In [1]:
from typing import Tuple

import itertools

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'japan'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params).fit(df)
    return optimal_model

In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

02:03:51 - cmdstanpy - INFO - Chain [1] start processing
02:03:51 - cmdstanpy - INFO - Chain [1] done processing
02:03:52 - cmdstanpy - INFO - Chain [1] start processing
02:03:52 - cmdstanpy - INFO - Chain [1] start processing
02:03:52 - cmdstanpy - INFO - Chain [1] start processing
02:03:53 - cmdstanpy - INFO - Chain [1] start processing
02:03:52 - cmdstanpy - INFO - Chain [1] start processing
02:03:52 - cmdstanpy - INFO - Chain [1] start processing
02:03:53 - cmdstanpy - INFO - Chain [1] start processing
02:03:53 - cmdstanpy - INFO - Chain [1] start processing
02:03:53 - cmdstanpy - INFO - Chain [1] done processing
02:03:53 - cmdstanpy - INFO - Chain [1] done processing
02:03:53 - cmdstanpy - INFO - Chain [1] done processing
02:03:53 - cmdstanpy - INFO - Chain [1] done processing
02:03:53 - cmdstanpy - INFO - Chain [1] done processing
02:03:53 - cmdstanpy - INFO - Chain [1] done processing
02:03:53 - cmdstanpy - INFO - Chain [1] start processing
02:03:53 - cmdstanpy - INFO - Chain [1

In [9]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [10]:
df_predicted, rmse = test_model(df_test, model)

In [11]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,2.218117,1.915299,2.517910,2.218117,2.218117,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.0,0.0,0.0,2.218165
1,2022-04-01,2.202064,1.890390,2.499408,2.202064,2.202064,-0.002691,-0.002691,-0.002691,-0.002691,-0.002691,-0.002691,0.0,0.0,0.0,2.199373
2,2022-05-01,2.186530,1.894164,2.473600,2.186530,2.186530,-0.004915,-0.004915,-0.004915,-0.004915,-0.004915,-0.004915,0.0,0.0,0.0,2.181614
3,2022-06-01,2.170477,1.873144,2.493495,2.170477,2.170477,0.015766,0.015766,0.015766,0.015766,0.015766,0.015766,0.0,0.0,0.0,2.186243
4,2022-07-01,2.154942,1.842317,2.480164,2.154942,2.154942,-0.003188,-0.003188,-0.003188,-0.003188,-0.003188,-0.003188,0.0,0.0,0.0,2.151754
5,2022-08-01,2.138889,1.833501,2.473686,2.138889,2.138889,0.010904,0.010904,0.010904,0.010904,0.010904,0.010904,0.0,0.0,0.0,2.149793
6,2022-09-01,2.122837,1.808241,2.438507,2.121804,2.122837,-0.001269,-0.001269,-0.001269,-0.001269,-0.001269,-0.001269,0.0,0.0,0.0,2.121568
7,2022-10-01,2.107302,1.785240,2.417989,2.102995,2.107309,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.0,0.0,0.0,2.107439
8,2022-11-01,2.091249,1.783232,2.388929,2.083664,2.093832,-0.005956,-0.005956,-0.005956,-0.005956,-0.005956,-0.005956,0.0,0.0,0.0,2.085293
9,2022-12-01,2.075714,1.744155,2.402463,2.063969,2.083995,-0.008959,-0.008959,-0.008959,-0.008959,-0.008959,-0.008959,0.0,0.0,0.0,2.066755


In [12]:
rmse

0.43083154210592867

In [13]:
with open(f'{country}_prophet_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [14]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [15]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [16]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,2.029109,1.725211,2.345856,2.029109,2.029109,0.009864,0.009864,0.009864,0.009864,0.009864,0.009864,0.0,0.0,0.0,2.038973
1,2023-04-01,2.013057,1.700719,2.293228,2.013057,2.013057,-0.004148,-0.004148,-0.004148,-0.004148,-0.004148,-0.004148,0.0,0.0,0.0,2.008909
2,2023-05-01,1.997522,1.667382,2.309184,1.997522,1.997522,-0.017753,-0.017753,-0.017753,-0.017753,-0.017753,-0.017753,0.0,0.0,0.0,1.979769
